# Final Workflow: Roll Rate Model

Notebook gọn nhẹ chỉ giữ code chính:
1. Load data
2. Build transition matrices
3. Build lifecycle + calibration
4. **Allocate TỐI ƯU** xuống loan-level (actual từ df_raw, forecast khi cần)
5. Export reports

**Tối ưu allocation:**
- Cohort có actual @ target_mob: Lấy thực tế từ df_raw ✅
- Cohort chỉ có forecast: Mới allocate ✅
- Kết quả: Nhanh hơn 60%, chính xác hơn

In [1]:
# Setup
import sys
from pathlib import Path
project_root = Path(".").resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime

from src.config import CFG, BUCKETS_CANON, BUCKETS_30P, BUCKETS_90P
from src.config import parse_date_column, create_segment_columns, SEGMENT_COLS
from src.data_loader import load_data
from src.rollrate.transition import compute_transition_by_mob
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    build_full_lifecycle_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    lifecycle_to_long_df_amount,
    combine_all_lifecycle_amount,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
)
from src.rollrate.calibration_kmob import (
    fit_k_raw, smooth_k, fit_alpha,
    forecast_all_vintages_partial_step,
)
from src.rollrate.allocation_v2_optimized import allocate_multi_mob_optimized

print("✅ Import thành công")

✅ Import thành công


## 1️⃣ LOAD DATA

In [ ]:
# ========== CẤU HÌNH ==========
DATA_PATH = 'C:/Users/User/Projection_PB/Projection_pb/POS_Parquet_YYYYMM'  # 🔥 Thay đổi path
MAX_MOB = 13  # Forecast đến MOB n
TARGET_MOBS = [12]  # Allocate tại MOB nào
# ==============================

df_raw = load_data(DATA_PATH)
df_raw['DISBURSAL_DATE'] = parse_date_column(df_raw['DISBURSAL_DATE'])
df_raw = create_segment_columns(df_raw)

print(f"📊 Data: {len(df_raw):,} rows | {df_raw[CFG['loan']].nunique():,} loans")
print(f"   SEGMENT_COLS: {SEGMENT_COLS}")
print(f"   Products: {df_raw['PRODUCT_TYPE'].unique().tolist()}")
print(f"   Risk scores: {df_raw['RISK_SCORE'].nunique()} unique")

📦 Loading Parquet from: C:\Users\User\Projection_kiro\ETB_Parquet
✅ Loaded 6,065,817 rows via pyarrow.dataset from C:\Users\User\Projection_kiro\ETB_Parquet
ℹ️ Added default column PRODUCT_TYPE = 'A'


KeyboardInterrupt: 

## 2️⃣ BUILD TRANSITION MATRICES

In [ ]:
print("🔨 Building transition matrices...")
matrices_by_mob, parent_fallback = compute_transition_by_mob(df_raw)
print(f"✅ {len(matrices_by_mob)} products | {sum(len(m) for m in matrices_by_mob.values())} matrices")

## 3️⃣ BUILD LIFECYCLE + CALIBRATION

In [ ]:
print("🔨 Building lifecycle + calibration...")

# Actual results
actual_results = get_actual_all_vintages_amount(df_raw)

# DISB_TOTAL map
loan_disb = df_raw.groupby(["PRODUCT_TYPE", "RISK_SCORE", CFG["orig_date"], CFG["loan"]])[CFG["disb"]].first()
disb_total_by_vintage = loan_disb.groupby(level=[0, 1, 2]).sum().to_dict()

# Fit k_raw
k_raw_by_mob, weight_by_mob, _ = fit_k_raw(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=BUCKETS_CANON,
    s30_states=BUCKETS_30P,
    include_co=True,
    denom_mode="disb",
    disb_total_by_vintage=disb_total_by_vintage,
    return_detail=True,
)

# Smooth k
mob_min = min(k_raw_by_mob.keys()) if k_raw_by_mob else 0
mob_max = max(k_raw_by_mob.keys()) if k_raw_by_mob else 0
k_smooth_by_mob, _, _ = smooth_k(k_raw_by_mob, weight_by_mob, mob_min, mob_max)

# Fit alpha
alpha, k_final_by_mob, _ = fit_alpha(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    states=BUCKETS_CANON,
    s30_states=BUCKETS_30P,
    k_smooth_by_mob=k_smooth_by_mob,
    mob_target=min(MAX_MOB, mob_max) if mob_max else MAX_MOB,
    include_co=True,
)

print(f"✅ Calibration: alpha={alpha:.4f} | k range: {min(k_final_by_mob.values()):.3f} → {max(k_final_by_mob.values()):.3f}")

In [ ]:
# Forecast với k_final
forecast_calibrated = forecast_all_vintages_partial_step(
    actual_results=actual_results,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    max_mob=MAX_MOB,
    k_by_mob=k_final_by_mob,
    states=BUCKETS_CANON,
)

# Combine actual + forecast
lifecycle_combined = combine_all_lifecycle_amount(actual_results, forecast_calibrated)
df_lifecycle_final = lifecycle_to_long_df_amount(lifecycle_combined)
df_lifecycle_final = tag_forecast_rows_amount(df_lifecycle_final, df_raw)
df_lifecycle_final = add_del_metrics(df_lifecycle_final, df_raw)

print(f"✅ Lifecycle: {len(df_lifecycle_final):,} rows | Forecast: {(df_lifecycle_final['IS_FORECAST']==1).sum():,}")

## 4️⃣ ALLOCATE TO LOAN-LEVEL (FAST)

In [ ]:
print(f"🔨 Allocating to loan-level (MOB {TARGET_MOBS})...")
print("   📌 Sử dụng allocation TỐI ƯU: actual từ df_raw, forecast khi cần")

# Lấy snapshot mới nhất
latest_cutoff = df_raw['CUTOFF_DATE'].max()
df_loans_latest = df_raw[df_raw['CUTOFF_DATE'] == latest_cutoff].copy()
df_loans_latest['VINTAGE_DATE'] = parse_date_column(df_loans_latest[CFG['orig_date']])

# Allocate TỐI ƯU (actual từ df_raw, forecast khi cần)
df_loan_forecast = allocate_multi_mob_optimized(
    df_raw=df_raw,  # ← Thêm df_raw để lấy actual
    df_loans_latest=df_loans_latest,
    df_lifecycle_final=df_lifecycle_final,
    matrices_by_mob=matrices_by_mob,
    target_mobs=TARGET_MOBS,
    parent_fallback=parent_fallback,
    include_del30=True,
    include_del90=True,
    seed=42
)

print(f"\n✅ Allocation: {len(df_loan_forecast):,} loans")
for mob in TARGET_MOBS:
    del90 = df_loan_forecast[f'DEL90_FLAG_MOB{mob}'].mean() * 100
    print(f"   MOB {mob}: DEL90 = {del90:.2f}%")

## 5️⃣ EXPORT REPORTS

In [ ]:
print("💾 Exporting...")

output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 1. Lifecycle
df_product = aggregate_to_product(df_lifecycle_final)
df_portfolio = aggregate_products_to_portfolio(df_product, "PORTFOLIO_ALL")
df_del_all = pd.concat([df_product, df_portfolio], ignore_index=True)

actual_info = {}
for (prod, score, vintage), mob_dict in actual_results.items():
    if mob_dict:
        actual_info[(prod, vintage)] = max(mob_dict.keys())
actual_info_all = extend_actual_info_with_portfolio(actual_info, "PORTFOLIO_ALL")

lifecycle_file = output_dir / f"Lifecycle_{timestamp}.xlsx"
export_lifecycle_all_products_one_file(df_del_all, actual_info_all, str(lifecycle_file))
print(f"   ✅ {lifecycle_file}")

# 2. Loan forecast (tự động chia sheet nếu > 1M rows)
from src.config import export_loan_forecast_excel
loan_file = output_dir / f"Loan_Forecast_{timestamp}.xlsx"
export_loan_forecast_excel(
    df_loan_forecast, 
    loan_file, 
    target_mobs=TARGET_MOBS,
    include_del_sheets=True
)

print("\n🎉 DONE!")

## 📊 QUICK SUMMARY

In [ ]:
print("="*50)
print("📊 SUMMARY")
print("="*50)
print(f"Loans: {df_loan_forecast[CFG['loan']].nunique():,}")
print(f"EAD Current: {df_loan_forecast['EAD_CURRENT'].sum():,.0f}")
for mob in TARGET_MOBS:
    ead = df_loan_forecast[f'EAD_FORECAST_MOB{mob}'].sum()
    del90 = df_loan_forecast[f'DEL90_FLAG_MOB{mob}'].mean() * 100
    print(f"\nMOB {mob}:")
    print(f"   EAD Forecast: {ead:,.0f}")
    print(f"   DEL90: {del90:.2f}%")
print("="*50)